#Classificação de Imagem

In [ ]:
import os
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import transforms, ToTensor, Resize, Compose
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from PIL import Image
import torch.nn.functional as F
from google.colab import drive

In [ ]:
# Verificando a disponibilidade de CPU ou GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
# Monta o Google Drive
drive.mount('/content/drive', force_remount=True)

# Muda o diretório para a pasta especificada no Google Drive
%cd /content/drive/MyDrive/DL_02_2024/MEU/Class_Imagens

Mounted at /content/drive
/content/drive/MyDrive/DL_02_2024/MEU/Class_Imagens


#Pre-Processamento

In [ ]:
num_skipped = 0  # Inicializa o contador de imagens corrompidas

for dataset in ("train", "test"):  # Itera sobre os conjuntos de dados de treino e teste
    dataset_path = os.path.join("Cars Dataset", dataset)  # Cria o caminho para o conjunto de dados atual

    for brand_folder in os.listdir(dataset_path):  # Itera sobre as pastas de marcas dentro do conjunto de dados
        folder_path = os.path.join(dataset_path, brand_folder)  # Cria o caminho para a pasta da marca atual

        for fname in os.listdir(folder_path):  # Itera sobre os arquivos dentro da pasta da marca
            fpath = os.path.join(folder_path, fname)  # Cria o caminho completo para o arquivo atual

            try:
                fobj = open(fpath, "rb")  # Abre o arquivo em modo binário de leitura
                is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)  # Verifica se o arquivo contém o marcador "JFIF" nos primeiros 10 bytes

            finally:
                fobj.close()  # Fecha o arquivo

            if not is_jfif:  # Se o arquivo não for um JFIF válido
                num_skipped += 1  # Incrementa o contador de imagens corrompidas
                os.remove(fpath)  # Remove o arquivo corrompido

print("Deleted %d images" % num_skipped)  # Imprime o número total de imagens deletadas

Deleted 0 images


#Treinamento em Modelo Simples

In [ ]:
# Defina o diretório raiz das pastas de treinamento e teste
root_dir = 'Cars Dataset'

# Nome das pastas de treinamento e teste
train_folder_name = 'train'
test_folder_name = 'test'

# Caminhos para as pastas de treinamento e teste
train_folder_path = os.path.join(root_dir, train_folder_name)
test_folder_path = os.path.join(root_dir, test_folder_name)

# Criando as pastas de treinamento e teste, se ainda não existirem
os.makedirs(train_folder_path, exist_ok=True)
os.makedirs(test_folder_path, exist_ok=True)

In [ ]:
# Caminhos para as pastas de treinamento e teste
train_folder = 'Cars Dataset/train'
test_folder = 'Cars Dataset/test'

# Primeiro, as imagens são convertidas para tensores (ToTensor), e depois normalizadas com média e desvio padrão de 0.5 para cada canal de cor (RGB).
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# As transformações definidas anteriormente são aplicadas às imagens.
train_dataset = ImageFolder(train_folder, transform=transform)
test_dataset = ImageFolder(test_folder, transform=transform)

# Criação dos data loaders de treinamento e teste
batch_size = 32  # Define o tamanho do lote
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Verificação da estrutura do dataset
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")
print(f"Classes: {train_dataset.classes}")

Number of training samples: 3348
Number of testing samples: 812
Classes: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']


In [ ]:
# Número de Épocas de treinamento
epocas = 10

# Criar uma pasta para salvar os modelos
folder_path = 'Modelos'
os.makedirs(folder_path, exist_ok=True)

In [ ]:
# Caminhos para as pastas de treinamento e teste
train_folder = 'Cars Dataset/train'
test_folder = 'Cars Dataset/test'

# Define as transformações a serem aplicadas às imagens
transform = Compose([
    Resize((224, 224)),  # Redimensiona todas as imagens para 224x224
    ToTensor()           # Converte a imagem para um tensor PyTorch
])

# Cria datasets para treinamento e teste
train_dataset = ImageFolder(train_folder, transform=transform)
test_dataset = ImageFolder(test_folder, transform=transform)

# Cria data loaders para treinamento e teste
batch_size = 32  # Define o tamanho do lote
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Verifica a estrutura do dataset
num_classes = len(train_dataset.classes)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")
print(f"Classes: {train_dataset.classes}")

# Define o modelo
class SimpleModel(nn.Module):
    def __init__(self, num_classes):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Primeira camada convolucional
        self.pool = nn.MaxPool2d(2, 2)   # Camada de pooling
        self.conv2 = nn.Conv2d(6, 16, 5) # Segunda camada convolucional
        self.fc1 = nn.Linear(16 * 53 * 53, 120)  # Primeira camada totalmente conectada
        self.fc2 = nn.Linear(120, 84)    # Segunda camada totalmente conectada
        self.fc3 = nn.Linear(84, num_classes)      # Terceira camada totalmente conectada

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Passa pela primeira camada convolucional e pooling
        x = self.pool(F.relu(self.conv2(x)))  # Passa pela segunda camada convolucional e pooling
        x = x.view(-1, 16 * 53 * 53)          # Achata o tensor para a camada totalmente conectada
        x = F.relu(self.fc1(x))               # Passa pela primeira camada totalmente conectada
        x = F.relu(self.fc2(x))               # Passa pela segunda camada totalmente conectada
        x = self.fc3(x)                       # Passa pela terceira camada totalmente conectada
        return x

model = SimpleModel(num_classes).to(device)

# Função de perda
criterion = nn.CrossEntropyLoss()

# Otimizador
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

# Função de treinamento (exemplo simplificado)
def treinamento():
    model.train()  # Coloca o modelo em modo de treinamento
    for epoch in range(epocas):  # Número de épocas
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # Zera os gradientes do otimizador
            outputs = model(images)  # Passa as imagens pelo modelo
            loss = criterion(outputs, labels)  # Calcula a perda
            loss.backward()  # Calcula os gradientes
            optimizer.step()  # Atualiza os pesos do modelo

            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Treinamento do modelo
treinamento()

# Salvar o modelo na pasta criada
folder_path = 'Cars Dataset/models'
os.makedirs(folder_path, exist_ok=True)
file_path = os.path.join(folder_path, 'modelosimples1.pth')
torch.save(model.state_dict(), file_path)

# Função para calcular a perda de entropia cruzada no conjunto de teste
def calcular_perda_teste():
    model.eval()  # Coloca o modelo em modo de avaliação
    test_loss = 0.0
    with torch.no_grad():  # Desabilita a computação do gradiente
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)  # Passa as imagens pelo modelo
            loss = criterion(outputs, labels)  # Calcula a perda
            test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)
    print(f'Perda de entropia cruzada no conjunto de teste: {avg_test_loss}')

# Calcule a perda de entropia cruzada no conjunto de teste
calcular_perda_teste()

Number of training samples: 3348
Number of testing samples: 812
Classes: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']
Epoch 1, Loss: 1.8390102113996234
Epoch 2, Loss: 1.628180301757086
Epoch 3, Loss: 1.2572632568223137
Epoch 4, Loss: 0.8889616248153505
Epoch 5, Loss: 0.5428347059658596
Epoch 6, Loss: 0.31636905343759625
Epoch 7, Loss: 0.1924787987910566
Epoch 8, Loss: 0.07612586427657377
Epoch 9, Loss: 0.11907439876702569
Epoch 10, Loss: 0.14057208705870877
Perda de entropia cruzada no conjunto de teste: 1.9936613417588747


#Treinamento em Modelo Complexo

In [ ]:
class DeepModel(nn.Module):
    def __init__(self):
        super(DeepModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)  # Primeira camada convolucional (entrada: 3 canais, saída: 16 canais, kernel: 5x5)
        self.pool = nn.MaxPool2d(2, 2)  # Camada de pooling (reduz a dimensão pela metade)
        self.bn1 = nn.BatchNorm2d(16)  # Normalização em lote para a primeira camada convolucional
        self.conv2 = nn.Conv2d(16, 32, 5)  # Segunda camada convolucional (entrada: 16 canais, saída: 32 canais, kernel: 5x5)
        self.bn2 = nn.BatchNorm2d(32)  # Normalização em lote para a segunda camada convolucional

        # Camadas totalmente conectadas (fully connected)
        self.fc1 = nn.Linear(32 * 53 * 53, 120)  # Primeira camada totalmente conectada
        self.fc2 = nn.Linear(120, 84)  # Segunda camada totalmente conectada
        self.fc3 = nn.Linear(84, 7)  # Terceira camada totalmente conectada (saída: 7 classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Aplicação da primeira camada convolucional, seguida de ReLU e pooling
        x = self.bn1(x)  # Aplicação da normalização em lote
        x = self.pool(F.relu(self.conv2(x)))  # Aplicação da segunda camada convolucional, seguida de ReLU e pooling
        x = self.bn2(x)  # Aplicação da normalização em lote
        x = torch.flatten(x, 1)  # Achatar o tensor para uma dimensão
        x = F.relu(self.fc1(x))  # Aplicação da primeira camada totalmente conectada com ReLU
        x = F.relu(self.fc2(x))  # Aplicação da segunda camada totalmente conectada com ReLU
        x = self.fc3(x)  # Aplicação da terceira camada totalmente conectada (saída)
        return x  # Retorna a saída final

model = DeepModel().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Treinamento do modelo
def treinamento():
    model.train()  # Coloca o modelo em modo de treinamento

    for epoch in range(epocas):  # Loop sobre o número de épocas
        running_loss = 0.0

        for images, labels in train_loader:  # Loop sobre o conjunto de dados de treinamento
            images = images.to(device)  # Move as imagens para o dispositivo (GPU ou CPU)
            labels = labels.to(device)  # Move os rótulos para o dispositivo

            optimizer.zero_grad()  # Zera os gradientes do otimizador
            outputs = model(images)  # Passa as imagens pelo modelo para obter as previsões
            loss = criterion(outputs, labels)  # Calcula a perda (loss) entre as previsões e os rótulos
            loss.backward()  # Calcula os gradientes da perda
            optimizer.step()  # Atualiza os pesos do modelo

            running_loss += loss.item()  # Acumula a perda para a época atual
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")  # Imprime a perda média por época

treinamento()

# Função para calcular a perda de entropia cruzada no conjunto de teste
def calcular_perda_teste():
    model.eval()  # Coloca o modelo em modo de avaliação
    test_loss = 0.0

    with torch.no_grad():  # Desabilita a computação do gradiente
        for images, labels in test_loader:  # Loop sobre o conjunto de dados de teste
            images = images.to(device)  # Move as imagens para o dispositivo
            labels = labels.to(device)  # Move os rótulos para o dispositivo
            outputs = model(images)  # Passa as imagens pelo modelo para obter as previsões
            loss = criterion(outputs, labels)  # Calcula a perda (loss) entre as previsões e os rótulos
            test_loss += loss.item()  # Acumula a perda para o conjunto de teste

    avg_test_loss = test_loss / len(test_loader)  # Calcula a perda média no conjunto de teste
    print(f'Perda de entropia cruzada no conjunto de teste: {avg_test_loss}')  # Imprime a perda média no conjunto de teste

# Calcule a perda de entropia cruzada no conjunto de teste
calcular_perda_teste()

# Salvar o modelo na pasta criada
folder_path = 'Cars Dataset/models'
os.makedirs(folder_path, exist_ok=True)
file_path = os.path.join(folder_path, 'modeloDeepModel2.pth')
torch.save(model.state_dict(), file_path)

Epoch 1, Loss: 1.5297067119961694
Epoch 2, Loss: 0.8231904495330084
Epoch 3, Loss: 0.33314117519628433
Epoch 4, Loss: 0.09618625945988156
Epoch 5, Loss: 0.03268284432235218
Epoch 6, Loss: 0.016293467000304235
Epoch 7, Loss: 0.010897889535962825
Epoch 8, Loss: 0.00813590305458222
Epoch 9, Loss: 0.00654165248519608
Epoch 10, Loss: 0.005314404306755889
Perda de entropia cruzada no conjunto de teste: 1.1393861805017178


#Treinamento em Modelo Complexo com múltiplos otimizadores

In [ ]:
model = DeepModel().to(device)

criterion = nn.CrossEntropyLoss()

# Função para treinamento
def treinamento(optimizer, epochs):
    model.train()  # Coloca o modelo em modo de treinamento
    for epoch in range(epochs):  # Loop sobre o número de épocas
        running_loss = 0.0  # Inicializa a perda acumulada para a época
        for images, labels in train_loader:  # Loop sobre os lotes de dados de treinamento
            images = images.to(device)  # Move as imagens para o dispositivo (CPU ou GPU)
            labels = labels.to(device)  # Move os rótulos para o dispositivo

            optimizer.zero_grad()  # Zera os gradientes do otimizador
            outputs = model(images)  # Passa as imagens pelo modelo para obter as previsões
            loss = criterion(outputs, labels)  # Calcula a perda entre as previsões e os rótulos verdadeiros
            loss.backward()  # Calcula os gradientes da perda em relação aos parâmetros do modelo
            optimizer.step()  # Atualiza os parâmetros do modelo com base nos gradientes

            running_loss += loss.item()  # Acumula a perda do lote atual
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")  # Imprime a perda média da época

# Função para calcular a perda de teste
def calcular_perda_teste():
    model.eval()  # Coloca o modelo em modo de avaliação
    test_loss = 0.0  # Inicializa a perda acumulada para o conjunto de teste
    with torch.no_grad():  # Desativa o cálculo de gradientes
        for images, labels in test_loader:  # Loop sobre os lotes de dados de teste
            images = images.to(device)  # Move as imagens para o dispositivo
            labels = labels.to(device)  # Move os rótulos para o dispositivo
            outputs = model(images)  # Passa as imagens pelo modelo para obter as previsões
            loss = criterion(outputs, labels)  # Calcula a perda entre as previsões e os rótulos verdadeiros
            test_loss += loss.item()  # Acumula a perda do lote atual

    avg_test_loss = test_loss / len(test_loader)  # Calcula a perda média no conjunto de teste
    print(f'Perda de entropia cruzada no conjunto de teste: {avg_test_loss}')  # Imprime a perda média do conjunto de teste

# Função para testar diferentes otimizadores
def testar_otimizadores(optimizers, epochs):
    for opt in optimizers:  # Loop sobre a lista de otimizadores
        print(f"\nTreinando com {opt.__class__.__name__}")  # Imprime o nome da classe do otimizador atual
        # Reinicializa os parâmetros do modelo
        model.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)
        optimizer = opt(model.parameters())  # Inicializa o otimizador com os parâmetros do modelo
        treinamento(optimizer, epochs)  # Treina o modelo com o otimizador atual
        calcular_perda_teste()  # Calcula e imprime a perda no conjunto de teste

# Lista de otimizadores a serem testados
optimizers = [
    lambda params: optim.Adam(params, lr=0.001, weight_decay=0.001),  # Otimizador Adam com taxa de aprendizado e decaimento de peso específicos
    lambda params: optim.SGD(params, lr=0.001, momentum=0.9),  # Otimizador SGD com taxa de aprendizado e momento específicos
    lambda params: optim.RMSprop(params, lr=0.001),  # Otimizador RMSprop com taxa de aprendizado específica
]

# Testar os otimizadores
testar_otimizadores(optimizers, epocas)  # Chama a função para testar os otimizadores com o número de épocas especificado


Treinando com function
Epoch 1, Loss: 1.7629346336637224
Epoch 2, Loss: 0.7161129390909559
Epoch 3, Loss: 0.20866118165708725
Epoch 4, Loss: 0.05241065383667037
Epoch 5, Loss: 0.01628165067328761
Epoch 6, Loss: 0.006454176151947606
Epoch 7, Loss: 0.0037361174677720383
Epoch 8, Loss: 0.004836687911301851
Epoch 9, Loss: 0.00536273560553257
Epoch 10, Loss: 0.42209279685990797
Perda de entropia cruzada no conjunto de teste: 2.898401750968053

Treinando com function
Epoch 1, Loss: 1.5381419215883527
Epoch 2, Loss: 0.8507307512419564
Epoch 3, Loss: 0.36886962382566363
Epoch 4, Loss: 0.1239974129767645
Epoch 5, Loss: 0.04000207536986896
Epoch 6, Loss: 0.018969126369449356
Epoch 7, Loss: 0.01249837643866028
Epoch 8, Loss: 0.009319849483047923
Epoch 9, Loss: 0.0070883393287658695
Epoch 10, Loss: 0.005770426318936405
Perda de entropia cruzada no conjunto de teste: 1.1484375366797814

Treinando com function
Epoch 1, Loss: 3.2340169656844364
Epoch 2, Loss: 1.180960364001138
Epoch 3, Loss: 0.71009

#Teste

In [ ]:
# Pasta onde o modelo foi salvo
model_path = 'Cars Dataset/models/modeloDeepModel2.pth'

# Carrega o modelo
model = DeepModel().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Define as transformações para a imagem de entrada
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Caminho da imagem que você deseja classificar
image_path = 'Cars Dataset/test/Audi/1000.jpg'

# Carrega a imagem
image = Image.open(image_path)
image = transform(image).unsqueeze(0).to(device)  # Adiciona uma dimensão de batch

# Faz a previsão
with torch.no_grad():
  output = model(image)
  _, predicted = torch.max(output, 1)

# Imprime a previsão
class_names = train_dataset.classes
predicted_class = class_names[predicted.item()]
print(f"A imagem foi classificada como: {predicted_class}")

<ipython-input-11-b34e0802f2cc>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


A imagem foi classificada como: Audi
